In [ ]:
# %pip install -qU langchain-nomic
# %pip install unstructured

In [ ]:
# !nomic login 'nk-nzSvRBdUz51LmQltAqVa3HXJ4epsEowE4ba8mbn56Fs'

## RAG

In [1]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# Load all text files from a folder
loader = DirectoryLoader("/weka/s223795137/Crawl_data/pages_datasets", glob="*.txt")  
documents = loader.load()


In [10]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


# # Load your text document
loader = TextLoader("website_text.txt")
documents = loader.load()

In [4]:
# for i in range(len(documents)):

#     documents[i].page_content = "search_document: " + documents[i].page_content

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size= 1200,chunk_overlap=0)

all_splits = text_splitter.split_documents(documents)

In [2]:
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings.base import Embeddings
import torch

class SentenceTransformerEmbeddings(Embeddings):
    def __init__(self, model_name="nomic-ai/nomic-embed-text-v1.5", device=None, batch_size=32):
        super().__init__()
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.model = SentenceTransformer(model_name, trust_remote_code=True, device=self.device)
        self.batch_size = batch_size  # Control batch size

    def embed_documents(self, texts):

        texts = ["search_document: "+i for i in texts]
        return self.model.encode(
            texts, 
            convert_to_numpy=True, 
            device=self.device, 
            batch_size=self.batch_size
        ).tolist()

    def embed_query(self, text):
        return self.model.encode(
            ['search_query: '+text], 
            convert_to_numpy=True, 
            device=self.device
        )[0].tolist()


# Initialize the embedding model
local_embeddings = SentenceTransformerEmbeddings(batch_size=64)

/home/s223795137/.conda/envs/llava/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
!!!!!!!!!!!!megablocks not available, using torch.matmul instead
<All keys matched successfully>


In [3]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

# local_embeddings = OllamaEmbeddings(model="nomic-embed-text")

# vectorstore = Chroma.from_documents(documents=all_splits, embedding=local_embeddings , persist_directory="./chroma_db_pages_veach_page")
# vectorstore = Chroma(persist_directory="./chroma_db_pages_v3_2.5k", embedding_function=local_embeddings)



vectorstore = Chroma(persist_directory="./chroma_db_pages_v3_2.5k")
# vectorstore = Chroma(persist_directory="./chroma_db_pages", embedding_function=local_embeddings)


In [9]:
??vectorstore.from_documents

Signature:
vectorstore.from_documents(
    documents: 'List[Document]',
    embedding: 'Optional[Embeddings]' = None,
    ids: 'Optional[List[str]]' = None,
    collection_name: 'str' = 'langchain',
    persist_directory: 'Optional[str]' = None,
    client_settings: 'Optional[chromadb.config.Settings]' = None,
    client: 'Optional[chromadb.ClientAPI]' = None,
    collection_metadata: 'Optional[Dict]' = None,
    **kwargs: 'Any',
) -> 'Chroma'
Source:   
    @classmethod
    def from_documents(
        cls: Type[Chroma],
        documents: List[Document],
        embedding: Optional[Embeddings] = None,
        ids: Optional[List[str]] = None,
        collection_name: str = _LANGCHAIN_DEFAULT_COLLECTION_NAME,
        persist_directory: Optional[str] = None,
        client_settings: Optional[chromadb.config.Settings] = None,
        client: Optional[chromadb.ClientAPI] = None,  # Add this line
        collection_metadata: Optional[Dict] = None,
        **kwargs: Any,
    ) -> Chroma:
   

In [4]:
# from langchain_ollama import ChatOllama

# model = ChatOllama(
#     model="/home-old/s223795137/models/models--meta-llama--Meta-Llama-3-8B-Instruct/snapshots/e1945c40cd546c78e41f1151f4db032b271faeaa",
# )

In [5]:
# model

In [6]:
# response_message = model.invoke(
#     "Simulate a rap battle between Stephen Colbert and John Oliver"
# )

# print(response_message.content)

In [ ]:
from langchain.llms import CTransformers
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline


# model_name="/home-old/s223795137/models/models--meta-llama--Meta-Llama-3-8B-Instruct/snapshots/e1945c40cd546c78e41f1151f4db032b271faeaa",  

model_name='microsoft/phi-4'

# model_name  = 'meta-llama/Llama-3.2-3B-Instruct'

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", device_map="auto")

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=4000)


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
llm = HuggingFacePipeline(pipeline=pipe)

/tmp/ipykernel_513427/2691217058.py:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [9]:
from langchain import LLMChain, PromptTemplate
from langchain.chains import RetrievalQA, StuffDocumentsChain


RAG_TEMPLATE = """<|im_start|>system<|im_sep|>
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. Try to Answer the questions based on Your understanding of the provided context texts , If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

<context>
{context}
</context>
<|im_end|>
<|im_start|>user<|im_sep|>
Answer the following question:

{question}

<|im_start|>assistant<|im_sep|>
"""


retriever = vectorstore.as_retriever(search_kwargs={"k": 5})  # Fetch top 5 chunks

# Define your prompt template
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""<|im_start|>system<|im_sep|><|im_end|>
<|im_start|>user<|im_sep|>Answer the question based on the context below:\n\nContext:\n{context}\n\nQuestion:\n{question}\n\nAnswer:<|im_start|>assistant<|im_sep|>""",
    # template = RAG_TEMPLATE
)

# Define your LLMChain
llm_chain = LLMChain(llm=llm, prompt=prompt_template)  # Replace with your LLM


stuff_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context"  # This should match the input variable in your prompt
)
# Combine into a RetrievalQA chain
rag_chain = RetrievalQA(
    retriever=retriever,
    combine_documents_chain=stuff_chain,
)


def extract_answers(answers):

    answers = answers.split("<|im_start|>assistant<|im_sep|>")[-1]

    return answers


# Function to query the RAG model
def query_rag_model(query):
    response = rag_chain.run(query)

    response = extract_answers(response)
    
    return response

/tmp/ipykernel_513427/2435010401.py:32: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt_template)  # Replace with your LLM
/tmp/ipykernel_513427/2435010401.py:35: LangChainDeprecationWarning: This class is deprecated. Use the `create_stuff_documents_chain` constructor instead. See migration guide here: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain/
  stuff_chain = StuffDocumentsChain(
/tmp/ipykernel_513427/2435010401.py:40: LangChainDeprecationWarning: This class is deprecated. Use the `create_retrieval_chain` constructor instead. See migration guide here: https://python.langchain.com/docs/versions/migrating_chains/retrieval_qa/
  rag_chain = RetrievalQA(


In [ ]:
# question = "i'm a new student and don't know how to activate my account?"

q2 = "Which alternate formats are available with Blackboard Ally?"

res =rag_chain.run(q2)

In [18]:
print(res)

<|im_start|>system<|im_sep|><|im_end|>
<|im_start|>user<|im_sep|>Answer the question based on the context below:

Context:
IT Help - How do I resolve Duo Prompt display issues related to iOS or macOS content restrictions? Skip to page content Welcome to Deakin, let's get started with your Digital Essentials! Get help with activating your Deakin IT account,  connecting to wifi (Eduroam), printing on campus, installing Microsoft 365 apps, and accessing software for your units. Click here to visit "Getting Started with your Digital Essentials". MORE Toggle navigation Log in Home Knowledge IT Help (Knowledge Base) Cyber Security - MFA IT Help - How do I resolve Duo Prompt display issues related to iOS or macOS content restrictions? BMP Created with Sketch. BMP ZIP Created with Sketch. ZIP XLS Created with Sketch. XLS TXT Created with Sketch. TXT PPT Created with Sketch. PPT PNG Created with Sketch. PNG PDF Created with Sketch. PD F JPG Created with Sketch. JPG GIF Created with Sketch. GIF 

In [14]:
# from langchain_core.output_parsers import StrOutputParser
# from langchain_core.prompts import ChatPromptTemplate

# prompt = ChatPromptTemplate.from_template(
#     "Summarize the main themes in these retrieved docs: {docs}"
# )


# # Convert loaded documents into strings by concatenating their content
# # and ignoring metadata
# def format_docs(docs):
#     return "\n\n".join(doc.page_content for doc in docs)


# chain = {"docs": format_docs} | prompt | llm | StrOutputParser()

# question = "i'm a new student and don't know how to activate my account?"

# docs = vectorstore.similarity_search(question)

# # print(chain.invoke(docs))

In [1]:
import pandas as pd


df =pd.read_csv("/weka/s223795137/Crawl_data/chat_history.txt", sep="\t")

In [2]:
df

,Time: 2025-02-20 11:00:50
0,User: How to run an even in a teaching space i...
1,Model: To run an event in a teaching space at ...
2,1. **Understand the Services Offered:**
3,- Digital Services manages a standard opera...
4,2. **Room Setups and Equipment:**
...,...
65,Model: To make the Duo app remember your crede...
66,1. **Sign in using Deakin's Single Sign-On (SS...
67,2. **Select 'Remember me':** Before approving ...
68,3. **Approve the Push Notification:** After se...


## Page Crawled

In [1]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter



# Load all text files from a folder
# loader = DirectoryLoader("/weka/s223795137/Crawl_data/crawled_pages", glob="*.txt",show_progress=True)  

# documents = loader.load()


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
# text_splitter = RecursiveCharacterTextSplitter(chunk_size= 2000,chunk_overlap=0)

# all_splits = text_splitter.split_documents(documents)

In [2]:
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings.base import Embeddings
import torch

class SentenceTransformerEmbeddings(Embeddings):
    def __init__(self, model_name="nomic-ai/nomic-embed-text-v1.5", device=None, batch_size=32):
        super().__init__()
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.model = SentenceTransformer(model_name, trust_remote_code=True, device=self.device)
        self.batch_size = batch_size  # Control batch size

    def embed_documents(self, texts):

        texts = ["search_document: "+i for i in texts]
        return self.model.encode(
            texts, 
            convert_to_numpy=True, 
            device=self.device, 
            batch_size=self.batch_size
        ).tolist()

    def embed_query(self, text):
        return self.model.encode(
            ['search_query: '+text], 
            convert_to_numpy=True, 
            device=self.device
        )[0].tolist()


# Initialize the embedding model
local_embeddings = SentenceTransformerEmbeddings(batch_size=64)

/home/s223795137/.conda/envs/llava/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
!!!!!!!!!!!!megablocks not available, using torch.matmul instead
<All keys matched successfully>


In [12]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_community.llms import VLLM



# vectorstore = Chroma.from_documents(documents=all_splits, embedding=local_embeddings , persist_directory="./citaion_chrome_db")


# vectorstore = Chroma(embedding_function=local_embeddings , persist_directory="./citaion_chrome_db")

# vectorstore = Chroma(persist_directory="./chroma_db_pages_v3_2.5k", embedding_function=local_embeddings)

vectorstore = Chroma(persist_directory="./chroma_db_pages_v3_2.5k", embedding_function=local_embeddings)



In [4]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline


model_name='microsoft/phi-4'

# model_name = 'microsoft/Phi-4-multimodal-instruct'


llm = VLLM(
        model=model_name,
        tensor_parallel_size=1,
        trust_remote_code=True,  # mandatory for hf models
        max_new_tokens=32000,
    )


# tokenizer = AutoTokenizer.from_pretrained(model_name)

# model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16 , attn_implementation='flash_attention_2' , trust_remote_code=True)

# pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=1024,device='cuda')

# llm = HuggingFacePipeline(pipeline=pipe)

INFO 03-07 11:33:10 config.py:510] This model supports multiple tasks: {'classify', 'embed', 'generate', 'reward', 'score'}. Defaulting to 'generate'.
INFO 03-07 11:33:10 llm_engine.py:234] Initializing an LLM engine (v0.6.6.post1) with config: model='microsoft/phi-4', speculative_config=None, tokenizer='microsoft/phi-4', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=16384, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=microsoft/phi-4, num_scheduler_s

Loading safetensors checkpoint shards:   0% Completed | 0/6 [00:00<?, ?it/s]


INFO 03-07 11:33:18 model_runner.py:1099] Loading model weights took 27.3875 GB
INFO 03-07 11:33:19 worker.py:241] Memory profiling takes 1.35 seconds
INFO 03-07 11:33:19 worker.py:241] the current vLLM instance can use total_gpu_memory (79.11GiB) x gpu_memory_utilization (0.90) = 71.20GiB
INFO 03-07 11:33:19 worker.py:241] model weights take 27.39GiB; non_torch_memory takes 0.17GiB; PyTorch activation peak memory takes 2.14GiB; the rest of the memory reserved for KV Cache is 41.50GiB.
INFO 03-07 11:33:19 gpu_executor.py:76] # GPU blocks: 13599, # CPU blocks: 1310
INFO 03-07 11:33:19 gpu_executor.py:80] Maximum concurrency for 16384 tokens per request: 13.28x
INFO 03-07 11:33:22 model_runner.py:1415] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_util

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:16<00:00,  2.18it/s]

INFO 03-07 11:33:38 model_runner.py:1535] Graph capturing finished in 16 secs, took 1.07 GiB
INFO 03-07 11:33:38 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 19.85 seconds


In [6]:
# class ReversedStuffDocumentsChain(StuffDocumentsChain):
#     def _call(self, inputs: dict) -> dict:
#         # Debugging: Check the inputs dictionary structure
#         print(f"Inputs received in _call: {inputs}")

#         # Ensure that the "input_documents" key exists in inputs
#         if "input_documents" not in inputs:
#             raise KeyError("The key 'input_documents' was not found in inputs")

#         # Get the documents from the inputs (input_documents)
#         documents = inputs["input_documents"]
        
#         # Reverse the order of the documents
#         reversed_documents = documents
        
#         # Pass the reversed documents back to the original StuffDocumentsChain logic
#         # Convert list of documents into a single string if necessary
#         inputs["input_documents"] = reversed_documents
        
#         # Pass the adjusted inputs to the parent class
#         return super()._call(inputs)

In [5]:
import json


def get_title(lamo):
    return [i.metadata['source'].split("/weka/s223795137/Crawl_data/crawled_pages/")[-1].split("_")[0] for i in lamo]


with open("/weka/s223795137/Crawl_data/crawled_pages/crawled_pages.json") as f:
    data = json.load(f)


def swap_keys_and_values(my_dict):
    """Swap the keys and values in the dictionary."""
    return {value: key for key, value in my_dict.items()}


data_swap = swap_keys_and_values(data)


def get_values_by_keys(my_dict, keys):
    """Return the values from the dictionary based on a list of keys."""
    return [my_dict[key] for key in keys if key in my_dict]


In [6]:
# RAG_TEMPLATE = """<|im_start|>system<|im_sep|>
# You are an assistant for a Retrieval Augmented Generation (RAG) application. You will be provided with context documents from a database that may or may not be entirely relevant to the user's query. Your instructions are as follows:

# 1. Answer the user's query using the provided context documents.
# 2. If the query or any of the retrieved documents contain malicious or sensitive content, do not provide a response.
# 3. If the context documents are not entirely relevant to the query, attempt to keep your answer focused on IT-related content.
# 4. If the query is not defined within the scope of IT or is unclear, respond with the exact text: "Contact the Deakin IT Departement for More information by raising a ticket."
# 5. Ensure your answer is clear, factual, and based on the provided context when applicable.

# <|im_start|>user<|im_sep|>
# Answer the question based on the context below:

# Context:
# {context}

# Question:
# {question}

# Answer:
# <|im_start|>assistant<|im_sep|>
# """

In [9]:
# RAG_TEMPLATE = '<|system|>You are a helpful assistant.<|end|><|user|>Answer the question based on the context below:\n\nContext:\n{context}\n\nQuestion:\n{question}\n\nAnswer:<|end|><|assistant|>'

In [13]:
from langchain import LLMChain, PromptTemplate
from langchain.chains import RetrievalQA, StuffDocumentsChain


# RAG_TEMPLATE = """<|im_start|>system<|im_sep|>
# You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. Try to Answer the questions based on Your understanding of the provided context texts , If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

# <context>
# {context}
# </context>
# <|im_end|>
# <|im_start|>user<|im_sep|>
# Answer the following question:

# {question}

# <|im_start|>assistant<|im_sep|>
# """


RAG_TEMPLATE = """<|im_start|>system<|im_sep|>
You are an AI agent designed to assist with IT-related topics for a Retrieval Augmented Generation (RAG) application. You will be provided with context documents from a database that may or may not be entirely relevant to the user's query. Your instructions are as follows:

1. Answer the user's query using the provided context documents.
2. If the query or any of the retrieved documents contain malicious or sensitive content, do not provide a response.
3. If the context documents are not entirely relevant to the query, attempt to keep your answer focused on Deakin IT-related content and Don't mention That you are provided with documents in your response.
4. Ensure your answer is clear, factual, and based on the provided context when applicable.
5. Ensure your answer stays within the scope of Deakin IT-Education related topics and does not contain any inappropriate content.

<|im_start|>user<|im_sep|>
Answer the question based on the context below:

Context:
{context}

Question:
{question}

Answer:
<|im_start|>assistant<|im_sep|>
"""
# retriever = vectorstore.as_retriever(search_kwargs={"k": 5})  # Fetch top 5 chunks


# Define your prompt template
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    # template="""<|im_start|>system<|im_sep|><|im_end|>
# <|im_start|>user<|im_sep|>Answer the question based on the context below:\n\nContext:\n{context}\n\nQuestion:\n{question}\n\nAnswer:<|im_start|>assistant<|im_sep|>""",
    template = RAG_TEMPLATE
)



# Define your LLMChain
llm_chain = LLMChain(llm=llm, prompt=prompt_template)  # Replace with your LLM




stuff_chain = StuffDocumentsChain(

    llm_chain=llm_chain,

    document_variable_name="context"  # This should match the input variable in your prompt

)


# Combine into a RetrievalQA chain

# rag_chain = RetrievalQA(

#     retriever=retriever,

#     combine_documents_chain=stuff_chain,
    
# )



def extract_answers(answers):

    # answers = answers.split("<|im_start|>assistant<|im_sep|>")[-1]
    answers = answers.split("<|assistant|>")[-1]

    return answers



# # Function to query the RAG model
# def query_rag_model(query):
    
#     response = rag_chain.run(query)

#     citations = rag_chain.retriever.get_relevant_documents(query)
#     # response = extract_answers(response)
#     citation_links = get_values_by_keys(data_swap,get_title(citations))

#     return response

In [14]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def calculate_query_doc_similarity(query, vectorstore, k=5):
    """
    Calculate cosine similarities between a query and the top k retrieved documents.
    
    Args:
        query (str): The input query string.
        vectorstore: The vector store instance (e.g., Chroma).
        k (int): Number of top documents to retrieve (default: 3).
    
    Returns:
        dict: Contains doc_similarity_pairs and average_similarity.
    """
    # Initialize retriever
    retriever = vectorstore.as_retriever(search_kwargs={"k": k})

    # Step 1: Get the query embedding
    embedding_function = vectorstore.embeddings  # Use the same embedding function
    query_embedding = embedding_function.embed_query(query)

    # Step 2: Retrieve top k documents
    retrieved_docs = retriever.get_relevant_documents(query)

    # Step 3: Recompute embeddings for retrieved documents
    doc_embeddings = []
    for doc in retrieved_docs:
        doc_embedding = embedding_function.embed_query(doc.page_content)
        doc_embeddings.append(doc_embedding)

    query_embedding = np.array(query_embedding).reshape(1, -1)

    doc_embeddings = np.array(doc_embeddings)
    
    similarities = cosine_similarity(query_embedding, doc_embeddings)[0] if doc_embeddings.size > 0 else np.array([])

    print(similarities)
    average_similarity = np.mean(similarities) if similarities.size > 0 else 0.0


    return average_similarity

In [15]:
q = 'how to activate my account?'

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# Step 1: Get the query embedding
embedding_function = vectorstore.embeddings  # Use the same embedding function

query_embedding = embedding_function.embed_query(q)

# Step 2: Retrieve top k documents
retrieved_docs = retriever.get_relevant_documents(q)

# # Step 3: Recompute embeddings for retrieved documents
# doc_embeddings = []
# for doc in retrieved_docs:
#     doc_embedding = embedding_function.embed_query(doc.page_content)
#     doc_embeddings.append(doc_embedding)

In [19]:
retriever.get_relevant_documents(q)

[]

In [12]:
def query_rag_model(query, k):
    # Update the retriever's 'k' value
    retriever = vectorstore.as_retriever(search_kwargs={"k": k})

    rag_chain = RetrievalQA(
        retriever=retriever,
        combine_documents_chain=stuff_chain
    )
    
    # Run the RAG pipeline
    response = rag_chain.run(query)
    k
    # Extract the answer
    response = extract_answers(response)

    citations = rag_chain.retriever.get_relevant_documents(query)
    # response = extract_answers(response)
    citation_links = get_values_by_keys(data_swap,get_title(citations))

    sim_score = calculate_query_doc_similarity(query , vectorstore , k)
    # Log interaction in JSON format

    return response ,citation_links , f"{sim_score:.4f}"


# def store_rating(query, response, rating):
#     """Stores the rating after the response is shown."""
    
#     log_interaction(query, response, rating)
#     return "Your feedback has been recorded! ✅"


# Gradio interface with a Slider for 'k'
def rag_interface(query, k=5):

    response,links , score  = query_rag_model(query, k)

    links = list(set(links))

    links = [
            (f"Document {i+1}", f"{links[i]}") for i in range(len(links))
        ]

# Create hyperlinks in HTML format
    links_markdown = "\n".join([f"[{title}]({url})" for title, url in links])
# Combine the text and links

    if (len(links) == 0 and float(score) < 0.5) or float(score) < 0.5 :
        
        response = response
    else:

        response = response+"\n\nReferences:\n\n" + links_markdown

    return response , score

In [13]:
res,score =  rag_interface('how to activate my account?')

print(res)

print("-"*10)

print(score)

/tmp/ipykernel_1297917/1259401095.py:5: LangChainDeprecationWarning: This class is deprecated. Use the `create_retrieval_chain` constructor instead. See migration guide here: https://python.langchain.com/docs/versions/migrating_chains/retrieval_qa/
  rag_chain = RetrievalQA(
/tmp/ipykernel_1297917/1259401095.py:11: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = rag_chain.run(query)
Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.45s/it, est. speed input: 83.74 toks/s, output: 77.61 toks/s]

[]
To activate your account, follow these general steps:

1. **Check Your Email**: After creating your account, you should have received an activation email. Check your inbox (and spam folder) for an email from Deakin IT, which will include an activation link or instructions.

2. **Click the Activation Link**: Open the email and click on the provided link to activate your account. Ensure you’re connected to the internet when you do this.

3. **Follow the Instructions**: The link should lead you to a page where you may need to confirm your email address or set a password. Complete the necessary steps.

4. **Troubleshooting**: If you do not receive the email or the link does not work, look for an option to resend the activation email. If problems persist, contact Deakin IT support for assistance.

For specific account activation steps associated with Deakin's IT systems, refer to their official support or IT services page.
----------
0.0000



/tmp/ipykernel_1297917/1259401095.py:16: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  citations = rag_chain.retriever.get_relevant_documents(query)


In [ ]:
import gradio as gr

iface = gr.Interface(

    fn=rag_interface,  # Function to call
    
    inputs=[  # List of inputs: query and k (slider)
        gr.Textbox(label="Query", placeholder="Enter your question here..."),
        gr.Slider(minimum=1, maximum=25, step=1, value=5, label="Number of Documents to Retrieve (k)"),
    ]
    ,
    
    outputs=[  # Two outputs: response (markdown) and score (text)
        
        gr.Markdown(label="Response"),

        gr.Textbox(label="Confidence Score"),

    ],

    
    title="# 📚 Deakin IT AI Help Desk",
    
    description="Enter your query and adjust 'k' to control the number of documents to retrieve for answering."

)


iface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7866
* Running on public URL: https://c6c9ad344445a1e167.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.31s/it, est. speed input: 6081.07 toks/s, output: 50.63 toks/s]


In [14]:
q2 = "My DUO client is not working. What to do?"
# 
# res =rag_chain.run(q2,metadata={"input_documents":documents,'soruce'})

In [15]:
lamo = rag_chain.retriever.get_relevant_documents(q2)

/tmp/ipykernel_191284/3931470044.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  lamo = rag_chain.retriever.get_relevant_documents(q2)


In [ ]:
## read json file /weka/s223795137/Crawl_data/crawled_pages/crawled_pages.json


    

In [ ]:
# get the keys based on the values from the list lamo



In [ ]:
!python citaion_file.py

/home/s223795137/.conda/envs/llava/lib/python3.10/site-packages/langchain/__init__.py:30: UserWarning: Importing LLMChain from langchain root module is no longer supported. Please use langchain.chains.LLMChain instead.
  warnings.warn(
/home/s223795137/.conda/envs/llava/lib/python3.10/site-packages/langchain/__init__.py:30: UserWarning: Importing PromptTemplate from langchain root module is no longer supported. Please use langchain_core.prompts.PromptTemplate instead.
  warnings.warn(
/weka/s223795137/Crawl_data/citaion_file.py:8: LangChainDeprecationWarning: Importing Chroma from langchain.vectorstores is deprecated. Please replace deprecated imports:

>> from langchain.vectorstores import Chroma

with new imports of:

>> from langchain_community.vectorstores import Chroma
You can use the langchain cli to **automatically** upgrade many imports. Please see documentation here <https://python.langchain.com/docs/versions/v0_2/>
  from langchain.vectorstores import Chroma
!!!!!!!!!!!!megab

## Docling

In [11]:
pip install langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.35
    Uninstalling langchain-core-0.3.35:
      Successfully uninstalled langchain-core-0.3.35
Note: you may need to restart the kernel to use updated packages.


In [1]:
import argparse
import json
import logging
import os
import glob
from pathlib import Path
# from docling.backend.pypdfium2_backend import PyPdfiumDocumentBackend
from docling.datamodel.base_models import InputFormat
from docling.document_converter import (
    DocumentConverter,
    PdfFormatOption,
    WordFormatOption,
)
from docling.pipeline.simple_pipeline import SimplePipeline
from docling.pipeline.standard_pdf_pipeline import StandardPdfPipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings.base import Embeddings
import torch
from langchain_chroma import Chroma


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
# _log = logging.getLogger(__name__)

In [32]:
# directory_path = '/weka/s223795137/Crawl_data/Docling/'
# file_patterns = ['*.txt', '*.html', '*.docx', '*.pdf']
# output_dir = 'climate'
# embedding_batch_size = 64

# vector_database = 'climate_db'

# # List to store all the files
# all_files = []

# # Loop through each pattern and retrieve matching files
# for pattern in file_patterns:
#     files = glob.glob(os.path.join(directory_path, '**', pattern), recursive=True)
#     all_files.extend(files)

# input_paths = all_files

# # Initialize DocumentConverter for supported formats
# doc_converter = DocumentConverter(
#     allowed_formats=[
#         InputFormat.PDF,
#         InputFormat.IMAGE,
#         InputFormat.DOCX,
#         InputFormat.HTML,
#         InputFormat.PPTX,
#         InputFormat.ASCIIDOC,
#         InputFormat.CSV,
#         InputFormat.MD,
#     ],
#     format_options={
#         InputFormat.PDF: PdfFormatOption(
#             pipeline_cls=StandardPdfPipeline,
#         ),
#         InputFormat.DOCX: WordFormatOption(
#             pipeline_cls=SimplePipeline
#         ),
#     },
# )

# # Function to handle text files separately
# def process_text_file(file_path: str) -> dict:
#     """Process a .txt file and return a simple result structure."""
#     try:
#         with open(file_path, 'r', encoding='utf-8') as f:
#             content = f.read()
#         # Mimic Docling's result structure
#         return {
#             "status": "success",
#             "filename": Path(file_path).name,
#             "document": content  # Fake method to match Docling's output

#         }
#     except Exception as e:
#         _log.error(f"Error processing text file {file_path}: {str(e)}")
#         return {
#             "status": "failed",
#             "filename": Path(file_path).name,
#             "error": str(e)
#         }

# # Separate text files from other formats
# txt_files = [f for f in input_paths if f.lower().endswith('.txt')]

# non_txt_files = [f for f in input_paths if not f.lower().endswith('.txt')]

# # Convert non-text files with Docling
# conv_results = []
# if non_txt_files:
#     conv_results.extend(doc_converter.convert_all(non_txt_files))

# # Process text files separately
# for txt_file in txt_files:
#     txt_result = process_text_file(txt_file)
#     conv_results.append(txt_result)

# def get_all_data(docs, file_name=output_dir):
#     out_path = Path(file_name)

#     # Check if the Output_documents directory exists, if not, create it
#     out_path.mkdir(parents=True, exist_ok=True)

#     for doc in docs:
#         if isinstance(doc, dict):
#             with (out_path / f"{doc['filename']}.md").open("w") as fp:
#                 fp.write(doc['document'])
#         else:
#             with (out_path / f"{doc.input.file.stem}.md").open("w") as fp:
#                 fp.write(doc.document.export_to_markdown())

# get_all_data(conv_results)

In [1]:
# loader = DirectoryLoader('/weka/s223795137/Crawl_data/climate/', glob="*.md")

# documents = loader.load()

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=0)

# all_splits = text_splitter.split_documents(documents)

# class SentenceTransformerEmbeddings(Embeddings):
#     def __init__(self, model_name="nomic-ai/nomic-embed-text-v1.5", device=None, batch_size=32):
#         super().__init__()
#         self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
#         self.model = SentenceTransformer(model_name, trust_remote_code=True, device=self.device)
#         self.batch_size = batch_size  # Control batch size

#     def embed_documents(self, texts):
#         texts = ["search_document: "+i for i in texts]
#         return self.model.encode(
#             texts, 
#             convert_to_numpy=True, 
#             device=self.device, 
#             batch_size=self.batch_size
#         ).tolist()

#     def embed_query(self, text):
#         return self.model.encode(
#             ['search_query: '+text], 
#             convert_to_numpy=True, 
#             device=self.device
#         )[0].tolist()

# # Initialize the embedding model
# local_embeddings = SentenceTransformerEmbeddings(batch_size=64)


In [ ]:
# vectorstore = Chroma.from_documents(documents=all_splits, embedding=local_embeddings , persist_directory='climate_db')

In [31]:
# !python Docling/docling.py --direcotry-path '/weka/s223795137/Crawl_data/Docling' --output_dir 'climate' 

In [1]:
# from langchain.llms import CTransformers
# from langchain.llms import HuggingFacePipeline
# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from sentence_transformers import SentenceTransformer
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings.base import Embeddings
import torch
from langchain import LLMChain, PromptTemplate
from langchain.chains import RetrievalQA, StuffDocumentsChain
# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.vectorstores import Chroma
import gradio as gr
import json
import os
from langchain_community.llms import VLLM




# model_name="/home-old/s223795137/models/models--meta-llama--Meta-Llama-3-8B-Instruct/snapshots/e1945c40cd546c78e41f1151f4db032b271faeaa",  

model_name='microsoft/phi-4'

# model_name  = 'meta-llama/Llama-3.2-3B-Instruct'

# tokenizer = AutoTokenizer.from_pretrained(model_name)

# model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)

# pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=32000,device='cuda')




# llm = HuggingFacePipeline(pipeline=pipe)



class SentenceTransformerEmbeddings(Embeddings):
    def __init__(self, model_name="nomic-ai/nomic-embed-text-v1.5", device=None, batch_size=32):
        super().__init__()
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.model = SentenceTransformer(model_name, trust_remote_code=True, device=self.device)
        self.batch_size = batch_size  # Control batch size

    def embed_documents(self, texts):
        texts = ["search_document: "+i for i in texts]
        return self.model.encode(
            texts, 
            convert_to_numpy=True, 
            device=self.device, 
            batch_size=self.batch_size
        ).tolist()

    def embed_query(self, text):
        return self.model.encode(
            ['search_query: '+text], 
            convert_to_numpy=True, 
            device=self.device
        )[0].tolist()




def extract_answers(answers):


    answers = answers.split("<|im_start|>assistant<|im_sep|>")[-1]

    return answers




import datetime

chat_history_file = "Climate-related Disclosures.json"



def log_interaction(query, response, filename="chat_history.json"):
    """Logs the user query and model response to a JSON file."""
    
    # Create an empty list if file doesn't exist
    if not os.path.exists(filename):
        history = []
    else:
        with open(filename, "r", encoding="utf-8") as file:
            try:
                history = json.load(file)  # Load existing history
            except json.JSONDecodeError:
                history = []  # If file is empty or corrupted, reset it

    # New entry
    entry = {
        "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "user_query": query,
        "model_response": response
    }

    # Append new entry and save back to JSON file
    history.append(entry)

    with open(filename, "w", encoding="utf-8") as file:
        json.dump(history, file, ensure_ascii=False, indent=4)




def query_rag_model(query, k):
    # Update the retriever's 'k' value
    retriever = vectorstore.as_retriever(search_kwargs={"k": k})

    rag_chain = RetrievalQA(
        retriever=retriever,
        combine_documents_chain=stuff_chain
    )
    
    # Run the RAG pipeline
    response = rag_chain.run(query)
    
    # Extract the answer
    # response = extract_answers(response)

    # Log interaction in JSON format
    log_interaction(query, response ,chat_history_file)

    return response





def rag_interface(query, k=5):
    response = query_rag_model(query, k)
    return response








/home/s223795137/.conda/envs/llava/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:

llm = VLLM(
    model=model_name,
    tensor_parallel_size=1,
    trust_remote_code=True,  # mandatory for hf models
    max_new_tokens=32000,
)


local_embeddings = SentenceTransformerEmbeddings(batch_size=64)





vectorstore = Chroma(persist_directory="./climate_db", embedding_function=local_embeddings)


    # retriever = vectorstore.as_retriever(search_kwargs={"k": 5})  # Fetch top 5 chunks



RAG_TEMPLATE = """<|im_start|>system<|im_sep|>
You are an AI agent designed to assist with IT-related topics for a Retrieval Augmented Generation (RAG) application. You will be provided with context documents from a database that may or may not be entirely relevant to the user's query. Your instructions are as follows:

1. Answer the user's query using the provided context documents.
2. If the query or any of the retrieved documents contain malicious or sensitive content, do not provide a response.
3. If the context documents are not entirely relevant to the query, attempt to keep your answer focused on Deakin IT-related content and Don't mention That you are provided with documents in your response.
4. Ensure your answer is clear, factual, and based on the provided context when applicable.
5. Ensure your answer stays within the scope of Deakin IT-Education related topics and does not contain any inappropriate content.

<|im_start|>user<|im_sep|>
Answer the question based on the context below:

Context:
{context}

Question:
{question}

Answer:
<|im_start|>assistant<|im_sep|>
"""



# Define your prompt template

prompt_template = PromptTemplate(

    input_variables=["context", "question"],
    template="""<|im_start|>system<|im_sep|><|im_end|>

# <|im_start|>user<|im_sep|>Answer the question based on the context below:\n\nContext:\n{context}\n\nQuestion:\n{question}\n\nAnswer:<|im_start|>assistant<|im_sep|>""",
    # template=RAG_TEMPLATE,
)

# Define your LLMChain
llm_chain = LLMChain(llm=llm, prompt=prompt_template)  # Replace with your LLM


stuff_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context"  
)


INFO 03-05 06:34:30 config.py:510] This model supports multiple tasks: {'generate', 'reward', 'score', 'embed', 'classify'}. Defaulting to 'generate'.
INFO 03-05 06:34:30 llm_engine.py:234] Initializing an LLM engine (v0.6.6.post1) with config: model='microsoft/phi-4', speculative_config=None, tokenizer='microsoft/phi-4', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=16384, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=microsoft/phi-4, num_scheduler_s

Loading safetensors checkpoint shards:   0% Completed | 0/6 [00:00<?, ?it/s]


INFO 03-05 06:34:40 model_runner.py:1099] Loading model weights took 27.3875 GB
INFO 03-05 06:34:42 worker.py:241] Memory profiling takes 1.33 seconds
INFO 03-05 06:34:42 worker.py:241] the current vLLM instance can use total_gpu_memory (79.11GiB) x gpu_memory_utilization (0.90) = 71.20GiB
INFO 03-05 06:34:42 worker.py:241] model weights take 27.39GiB; non_torch_memory takes 0.17GiB; PyTorch activation peak memory takes 2.14GiB; the rest of the memory reserved for KV Cache is 41.50GiB.
INFO 03-05 06:34:42 gpu_executor.py:76] # GPU blocks: 13598, # CPU blocks: 1310
INFO 03-05 06:34:42 gpu_executor.py:80] Maximum concurrency for 16384 tokens per request: 13.28x
INFO 03-05 06:34:44 model_runner.py:1415] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_util

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:18<00:00,  1.89it/s]

INFO 03-05 06:35:02 model_runner.py:1535] Graph capturing finished in 19 secs, took 1.07 GiB
INFO 03-05 06:35:02 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 22.13 seconds



!!!!!!!!!!!!megablocks not available, using torch.matmul instead
<All keys matched successfully>
/tmp/ipykernel_1134353/3445714857.py:15: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory="./climate_db", embedding_function=local_embeddings)
/tmp/ipykernel_1134353/3445714857.py:58: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt_template)  # Replace with your LLM
/tmp/ipykernel_1134353/3445714857.py:61: LangChainDeprecationWarning: This class is deprecated. Use the `create_stuff_documents_chain

In [5]:
iface = gr.Interface(
# gr.Markdown("# 📚 Deakin IT AI Help Desk")

fn=rag_interface,   # Function to call
inputs=[            # List of inputs: query and k (slider)
    gr.Textbox(label="Query", placeholder="Enter your question here..."),
    gr.Slider(minimum=1, maximum=25, step=1, value=5, label="Number of Documents to Retrieve (k)"),


], 
outputs="text",     # Output type (text box)
title="# 📚 Climate-related Disclosures",
description="Enter your query and adjust 'k' to control the number of documents to retrieve for answering."
)


iface.launch(share=True)


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://7892231cfaa4b7d9f8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.99s/it, est. speed input: 333.37 toks/s, output: 77.50 toks/s]


In [1]:
!gradio gradio_run.py


Watching: '/weka/s223795137/Crawl_data' '/weka/s223795137/Crawl_data'

/weka/s223795137/.conda/envs/llava/lib/python3.10/site-packages/langchain/__init__.py:30: UserWarning: Importing LLMChain from langchain root module is no longer supported. Please use langchain.chains.LLMChain instead.
  warnings.warn(
/weka/s223795137/.conda/envs/llava/lib/python3.10/site-packages/langchain/__init__.py:30: UserWarning: Importing PromptTemplate from langchain root module is no longer supported. Please use langchain_core.prompts.PromptTemplate instead.
  warnings.warn(
/weka/s223795137/Crawl_data/gradio_run.py:11: LangChainDeprecationWarning: Importing Chroma from langchain.vectorstores is deprecated. Please replace deprecated imports:

>> from langchain.vectorstores import Chroma

with new imports of:

>> from langchain_community.vectorstores import Chroma
You can use the langchain cli to **automatically** upgrade many imports. Please see documentation here <https://python.langchain.com/docs/versio